# Part 1

In [60]:
with open('input_day24.txt') as file:
    lines = file.readlines()
    
    variables = dict()
    gates = []
    for line in lines:
        if ":" in line:
            tokens = line.strip().split(': ')
            name = tokens[0]
            value = int(tokens[1])
            variables[name] = value
        elif "->" in line:
            tokens = line.strip().split(' ')
            left = tokens[0]
            operator = tokens[1]
            right = tokens[2]
            output_var = tokens[4]
            gates.append((left, operator, right, output_var))

In [61]:
import numpy as np
gate_status = np.zeros(len(gates)).astype(bool)
def calculate(gates, variables):
    while gate_status.all() == False:
        for index,g in enumerate(gates):
            if gate_status[index] == False:
                left, operator, right, output_var = g
#                 print(g)
                if (left in variables) and (right in variables):
                    if operator == 'AND':
                        variables[output_var] = variables[left] & variables[right]
                    elif operator == 'OR':
                        variables[output_var] = variables[left] | variables[right]
                    elif operator == 'XOR':
                        variables[output_var] = variables[left] ^ variables[right]
                    gate_status[index] = True

In [62]:
calculate(gates, variables)
z_variables = dict()
for key, value in variables.items():
    if key.startswith('z'):
        z_variables[key] = value
sorted_dict = dict(sorted(z_variables.items(),  reverse = True))
final_binary = "".join([str(c) for c in sorted_dict.values()])
print(final_binary)
print('Answer for part 1:', int(final_binary, 2))

1100101000010010000000110000000111000011101000
Answer for part 1: 55544677167336


# Part 2

In [68]:
with open('input_day24.txt') as file:
    lines = file.readlines()
    
    variables = dict()
    gates = []
    for line in lines:
        if ":" in line:
            tokens = line.strip().split(': ')
            name = tokens[0]
            value = int(tokens[1])
            variables[name] = value
        elif "->" in line:
            tokens = line.strip().split(' ')
            left = tokens[0]
            operator = tokens[1]
            right = tokens[2]
            output_var = tokens[4]
            gates.append((left, operator, right, output_var))

In [108]:
# Ripple-carry adder: to add two integers 
# All XOR gates that input x__ and y__ cannot every output z__ (unless x00,y00 because the first one is a half adder)

# All other XOR gates must output z__

# All gates that output z__ must be XOR (except for z45, which is the final carry)

# All gates checked in (1) must output to gate checked in (2)

# If there are any swaps unaccounted for, manually review

# This solution is from @lscddit
# https://www.reddit.com/r/adventofcode/comments/1hl698z/2024_day_24_solutions/

wires = {}
operations = []

def process(op, op1, op2):
    if op == "AND":
        return op1 & op2
    elif op == "OR":
        return op1 | op2
    elif op == "XOR":
        return op1 ^ op2

highest_z = "z00"
data = open("input_day24.txt").read().split("\n")
for line in data:
    if ":" in line:
        wire, value = line.split(": ")
        wires[wire] = int(value)
    elif "->" in line:
        op1, op, op2, _, res = line.split(" ")
        operations.append((op1, op, op2, res))
        if res[0] == "z" and int(res[1:]) > int(highest_z[1:]):
            highest_z = res

wrong = set()
for op1, op, op2, res in operations:
    if res[0] == "z" and op != "XOR" and res != highest_z:
        wrong.add(res)
    if (
        op == "XOR"
        and res[0] not in ["x", "y", "z"]
        and op1[0] not in ["x", "y", "z"]
        and op2[0] not in ["x", "y", "z"]
    ):
        wrong.add(res)
    if op == "AND" and "x00" not in [op1, op2]:
        for subop1, subop, subop2, subres in operations:
            if (res == subop1 or res == subop2) and subop != "OR":
                wrong.add(res)
    if op == "XOR":
        for subop1, subop, subop2, subres in operations:
            if (res == subop1 or res == subop2) and subop == "OR":
                wrong.add(res)

while len(operations):
    op1, op, op2, res = operations.pop(0)
    if op1 in wires and op2 in wires:
        wires[res] = process(op, wires[op1], wires[op2])
    else:
        operations.append((op1, op, op2, res))

bits = [str(wires[wire]) for wire in sorted(wires, reverse=True) if wire[0] == "z"]
print('Answer for part 1:', int("".join(bits), 2))
print('Answer for part 2:', ",".join(sorted(wrong)))

Answer for part 1: 55544677167336
Answer for part 2: gsd,kth,qnf,tbt,vpm,z12,z26,z32
